In [1]:
from google.colab import files 
uploaded = files.upload()

Saving amazon_alexa.csv to amazon_alexa.csv


In [2]:
import pandas 
df_review = pandas.read_csv('amazon_alexa.csv', sep='\t')
df_review.head()


,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [4]:
def processRow(row):
 import re
 import nltk
 from textblob import TextBlob
 from nltk.corpus import stopwords
 from nltk.stem import PorterStemmer
 from textblob import Word
 from nltk.util import ngrams
 nltk.download('wordnet')
 import re
 from nltk.tokenize import word_tokenize
 tweet = row

#Lower case
 tweet.lower()
#Removes unicode strings like "\u002c"  -> ,(comma)
 tweet = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', tweet)
    
# Removes non-ascii characters. note : \x00 to \x7f is 00 to 255
# non-ascii characters like copyrigth symbol, trademark symbol
 tweet = re.sub(r'[^\x00-\x7f]',r'',tweet)
               
#convert any url to URL
 tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
               
#Convert any @Username to "AT_USER"
 tweet = re.sub('@[^\s]+','AT_USER',tweet)

#Remove additional white spaces
 tweet = re.sub('[\s]+', ' ', tweet)
 tweet = re.sub('[\n]+', ' ', tweet)
#Remove not alphanumeric symbols white spaces
 tweet = re.sub(r'[^\w]', ' ', tweet)

#Removes hastag in front of a word """
 tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

#Replace #word with word
 tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

#Removes all possible emoticons
 tweet = re.sub(':\)|:\(|:\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', tweet)

#remove numbers -> this is optional
 tweet = ''.join([i for i in tweet if not i.isdigit()])

#remove multiple exclamation -> this is optional
 tweet = re.sub(r"(\!)\1+", ' ', tweet)
#remove multiple question marks -> this is optional
 tweet = re.sub(r"(\?)\1+", ' ', tweet)

#remove multistop -> this is optional
 tweet = re.sub(r"(\.)\1+", ' ', tweet)

#trim
 tweet = tweet.strip('\'"')
     
#lemma
 from textblob import Word
 tweet =" ".join([Word(word).lemmatize() for word in tweet.split()])

#stemmer
#st = PorterStemmer()
#tweet=" ".join([st.stem(word) for word in tweet.split()])
          
 row = tweet
 return row


In [5]:
# clean your verified_reviews
cleaned_verified_reviews = []

for line in df_review['verified_reviews'] :
    cleanLine = processRow(line) 
    cleaned_verified_reviews.append(cleanLine)
    
import numpy as np    
df_review['cleaned_verified_reviews'] = np.asarray(cleaned_verified_reviews)

df_review.head(5)


Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading packa

,rating,date,variation,verified_reviews,feedback,cleaned_verified_reviews
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1,Love my Echo
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1,Loved it
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...
4,5,31-Jul-18,Charcoal Fabric,Music,1,Music


In [6]:
from textblob import TextBlob
# Let's define our sentiment analyzer function:
def analyze_sentiment(cleaned_verified_reviews):
    analysis = TextBlob(cleaned_verified_reviews)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'


In [7]:
# Lets find the Sentiment by calling the above defn fn.
# create a new column called 'Sentiment'
df_review['Sentiment'] = df_review['cleaned_verified_reviews'].apply(lambda x: analyze_sentiment(x))

df_review[['cleaned_verified_reviews', 'Sentiment']].head(3)


,cleaned_verified_reviews,Sentiment
0,Love my Echo,Positive
1,Loved it,Positive
2,Sometimes while playing a game you can answer ...,Neutral


In [8]:
# no. of positive sentiment reviews
df_review[df_review.Sentiment=='Positive'].shape[0]


2578

In [9]:
df_review[df_review.Sentiment=='Negative'].shape[0]

189

In [10]:
df_review[df_review.Sentiment=='Neutral'].shape[0]

383

In [12]:
# lets add polarity also to the df
df_review['polarity'] = df_review['cleaned_verified_reviews'].map(lambda text: TextBlob(text).sentiment.polarity)

df_review[['cleaned_verified_reviews', 'Sentiment', 'polarity']].head(10)


,cleaned_verified_reviews,Sentiment,polarity
0,Love my Echo,Positive,0.500000
1,Loved it,Positive,0.700000
2,Sometimes while playing a game you can answer ...,Neutral,0.000000
3,I have had a lot of fun with this thing My yr ...,Positive,0.233333
4,Music,Neutral,0.000000
5,I received the echo a a gift I needed another ...,Positive,0.294643
6,Without having a cellphone I cannot use many o...,Positive,0.216667
7,I think this is the th one I ve purchased I m ...,Positive,0.300000
8,look great,Positive,0.800000
9,Love it Ive listened to song I havent heard si...,Positive,0.650000
